# Study case: the 2nd classified solution of the challenge

Ideally in this notebook I will do 3 things:
1) study one of the top solutions of the challenge
2) replicate the feature engineering 
3) implement a version of the solution, as well as other models such as a DNN

## 2nd classified solution

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBClassifier
from xgboost import plot_importance

c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


A few global variables are set. One is the path to the training and test data, the other is a state to remove randomness of some algorithms (for reproduction purposes). In the same cell, we define the features and the target variables.

In [2]:
data_dir = '../Datasets/killer-shrimp-invasion/'
RANDOM_STATE = 0

train = pd.read_csv(data_dir + 'train.csv')
test = pd.read_csv(data_dir + 'test.csv')

# split data
Y_train = train['Presence']
ID_train = train['pointid']
X_train = train.drop(['Presence', 'pointid'], axis=1)
ID_test = test['pointid']
X_test = test.drop(['pointid'], axis=1)

### Missing data
As I previously attested in my own notebook, there is quite a lot of missing data. Using `sklearn`'s `IterativeImputer` we can impute values for missing data. It performs better than filling in with the mean. However, I'm lazy and I'm perfectly fine filling it with the mean...

In [3]:
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

### Feature Engineering

There are 5 features in the training data: temperature, salinity, depth, exposure and subtrate. WE create more features:
- Ocean Density: function of temperature, salinity and depth based on UNESCO formula
- EUNIS Exposure Classifications (nine features): this is a categorical based on exposure
- Temperature Equation: this is basically an equation obtained from fitting the training data using only the temperature as a feature. We use the response of this fitted equation as additional feature.
- Outlier (Bounded-Box): this are the conditions of the normal habitat of Killer Shrimps hard coded into the problem.
    - tolerate salinity up to 20ppt (12ppt is optimal)
    - tolerate temperatures between 0 and 35C (5-15 optimal)
    - thought to occupy every substratum except sand
    - present in areas of low current velocity

In [13]:
# Ocean density:
def ocean_density(temp, salin, depth):
    pressure_approx = 1 + (depth / 10)

    # standard mean ocean water density (smow)
    smow = 999.842594 + (6.793953e-2 * temp) - (9.095290e-3 * (temp**2)) + \
           (1.001685e-4 * (temp**3)) - (1.120083e-6 * (temp**4)) + (6.536332e-9 * (temp**5))

    # sea water density at normal atmospheric pressure
    B = 0.82449 - (4.0899e-3 * temp) + (7.6438e-5 * (temp**2)) + (8.2467e-7 * (temp**3)) + \
        (5.3875e-9 * (temp**4))
    C = -5.7246e-3 + (1.0227e-4 * temp) + (-1.6546e-6 * (temp**2))
    D = 4.8314e-4
    density_normal = smow + (B * salin) + (C * (salin**1.5)) + (D * (salin**2))

    # determination of compression module at pression 0
    Kw = 19652.21 + (148.4206 * temp) + (-2.327105 * (temp**2)) + (1.360477e-2 * (temp**3)) + \
        (-5.155288e-5 * (temp**4))
    F = 52.6746 + (-0.603459 * temp) + (1.099870e-2 * (temp**2)) + (-6.167e-5 * (temp**3))
    G = 7.9440e-2 + (1.6483e-2 * temp) + (-5.3009e-4 * (temp**2))
    K_0 = Kw + (F * salin) + (G * (salin**1.5))

    # determination of final compressibility module
    Aw = 3.23990 + (1.43713e-3 * temp) + (1.16092e-4 * (temp**2)) + (-5.77905e-7 * (temp**3))
    A1 = Aw + ((2.28380e-3 + (-1.09810e-5 * temp) + (1-60780e-6 * (temp**2))) * salin) + (1.91075e-4 * (salin ** 1.5))
    Bw = 8.50935e-5 + (-6.12293e-6 * temp) + (5.27870e-8 * (temp**2))
    B2 = Bw + (-9.9348e-7 + (2.0816e-8 * temp) + (9.1697e-10 * (temp**2))) * salin 
    K = K_0 + (A1*pressure_approx) + (B2*(pressure_approx**2))

    return (density_normal / (1- (pressure_approx / K))) - 1000

In [14]:
X_train['Density'] = X_train.apply(lambda x: ocean_density(x['Temperature_today'], x['Salinity_today'], x['Depth']), axis=1)
X_test['Density'] = X_test.apply(lambda x: ocean_density(x['Temperature_today'], x['Salinity_today'], x['Depth']), axis=1)

In [22]:
# EUNIS Exposure Classifications

def eunis_classification(exposure):
    if exposure <= 1200:
        return 'Ultra Sheltered'
    elif exposure <= 4000:
        return 'Extremely Sheltered'
    elif exposure <= 10000:
        return 'Very Sheltered'
    elif exposure <= 100000:
        return 'Sheltered'
    elif exposure <= 500000:
        return 'Moderately Exposed'
    elif exposure <= 1000000:
        return 'Exposed'
    elif exposure <= 2000000:
        return 'Very Exposed'
    else:
        return 'Extremely Exposed'

for classifi in ['Ultra Sheltered', 'Extremely Sheltered', 'Very Sheltered', 'Sheltered', 
                 'Moderately Exposed', 'Exposed', 'Very Exposed']:
    X_train['EUNIS ' + classifi] = X_train['Exposure'].apply(lambda x: 1 if eunis_classification(x) == classifi else 0)
    X_test['EUNIS ' + classifi] = X_test['Exposure'].apply(lambda x: 1 if eunis_classification(x) == classifi else 0)

X_train['EUNIS Min Very Exposed'] = X_train['Exposure'].apply(lambda x: 1 if eunis_classification(x) == 'Extremely Exposed' 
                                                              or eunis_classification(x) == 'Very Exposed' else 0)
X_test['EUNIS Min Very Exposed'] = X_test['Exposure'].apply(lambda x: 1 if eunis_classification(x) == 'Extremely Exposed' 
                                                              or eunis_classification(x) == 'Very Exposed' else 0)

In [16]:
# temperature model
def temperature_equation(t):
    coefs = [-10.52910058, 2.21529641, -0.57623745, -2.06832485, 0.30713969, -2.21781761, -1.85363275]
    temps = [1, t, t**2, t**3, t**4, t**5, t**6]
    summed = np.dot(coefs, temps)
    return 1 / (1 + np.exp(summed))

X_train['Temperature Model'] = X_train['Temperature_today'].apply(lambda x: temperature_equation(x))
X_test['Temperature Model'] = X_test['Temperature_today'].apply(lambda x: temperature_equation(x))

In [17]:
# Outliers (bounded box)
class OutlierFeature():
    def __init__(self, ratio, outlier_features):
        self.ratio = ratio
        self.outlier_features = outlier_features
    
    def exclude_outliers(self, row):
        lower_multiplier = self.ratio
        upper_multiplier = 1 / self.ratio
        conditions = []
        for feature, cmin, cmax in self.outlier_conditions:
            conditions.append(row[feature] > cmin)
            conditions.append(row[feature] < cmax)
        return 1 if sum(conditions) == len(conditions) else 0

    def fit(self, x, y):
        self.outlier_conditions = []
        for feature in self.outlier_features:
            fmin = x[y==1][feature].min()
            fmax = x[y==1][feature].max()
            lower_multiplier = self.ratio
            upper_multiplier = 1 / self.ratio

            cmin = fmin * lower_multiplier if fmin > 0 else fmin * upper_multiplier
            cmax = fmax * upper_multiplier if fmax > 0 else fmax * lower_multiplier
            self.outlier_conditions.append((feature, cmin, cmax))

    def transform(self, x):
        outliers = x.apply(lambda x: self.exclude_outliers(x), axis = 1)
        return outliers

outlier_finder = OutlierFeature(0.82, ['Temperature_today', 'Salinity_today', 'Exposure', 'Depth'])
outlier_finder.fit(X_train, Y_train)

X_train['Outlier'] = outlier_finder.transform(X_train)
X_test['Outlier'] = outlier_finder.transform(X_test)

## Models
In the next few steps, we define a function allowing to perform statified cross validation. This consists in evaluating cross-validation with stratified sampling (that is, sampling performed making sure that the percentage of a certain value is the same as in the dataset). For this, 5-fold cross-validation is used. It also prints out the AUROC score. The ROC curve is the plot obtained by plotting the True Positive Rate (sensitivity) to the false positive rate (1 - specificity), and AUROC is the Area Under the Receiver Operating Characteristic. If the area is 1, the model is perfect. If AUROC = 0.5, it operates like a random model.

In [4]:
def five_fold_cv(model, X_train, Y_train, verbose = True):
    skf = StratifiedKFold(n_splits = 5)
    fold = 1
    scores = []

    for train_index, test_index in skf.split(X_train, Y_train):
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]

        model.fit(X_train_fold, Y_train_fold)
        
        preds = model.predict_proba(X_test_fold)
        preds = [x[1] for x in preds]

        score = roc_auc_score(Y_test_fold, preds)
        scores.append(score)
        if verbose:
            print('Fold', fold, '    ', score)
        fold += 1

    avg = np.mean(scores)
    if verbose:
        print()
        print('Average', avg)
    return avg

Let's also define the list of features:

In [19]:
features = ['Temperature_today', 'Salinity_today', 'Depth', 'EUNIS Ultra Sheltered', 'EUNIS Extremely Sheltered', 
            'EUNIS Very Sheltered', 'EUNIS Sheltered', 'EUNIS Moderately Exposed', 'EUNIS Exposed', 'EUNIS Very Exposed', 
            'Substrate', 'Outlier', 'Temperature Model', 'Density', 'EUNIS Min Very Exposed']

Time to generate an XGB model (eXtreme Gradient Boosting). Gradient boosting is a ML technique used in regression and classification tasks. It gives a prediction model in the form of an ensemble of weak prediction models (typically, decision trees).

In [5]:
model = XGBClassifier(random_state = RANDOM_STATE, eval_metric = 'auc', objective = 'binary:logistic',
                      learning_rate = 0.3, max_depth = 5, subsample = 1, reg_lambda = 0.5)


In [25]:

score = five_fold_cv(model, X_train[features], Y_train, verbose=True)

c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Fold 1      0.9996289025683597


c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Fold 2      0.9996640321097338


c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Fold 3      0.9987628498911841


c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Fold 4      0.9994979046037786


c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Fold 5      0.9994874314068218

Average 0.9994082241159756


## Predictions

In [26]:
preds = pd.DataFrame(ID_test, columns=['pointid'])

model.fit(X_train[features], Y_train)
preds['Presence'] = model.predict_proba(X_test[features])[:,1]

#preds[['pointid', 'Presence']].to_csv('preds.csv', index=False)

c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\xtomma\Anaconda3\envs\machinelearning\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


This was a very interesting result! Now in another notebook I will look at the same approach without feature engineering and post it to gauge how much it can be gained really from it. Feature engineering was very cumbersome, and it might be worth it for minimal gains, but not really for giving a scientifically useful answer...